In [11]:
import http.client
import json
import time
import random
import os

In [2]:
def send_request(method, path, api_key, body=None):
    """Establishes a connection and sends a request to the server."""
    conn = http.client.HTTPSConnection("demo.imagineapi.dev")
    headers = {
        'Authorization': f'Bearer {api_key}',  # Authorisation header with API key
        'Content-Type': 'application/json'
    }
    conn.request(method, path, body=json.dumps(body) if body else None, headers=headers)
    response = conn.getresponse()
    data = json.loads(response.read().decode())
    conn.close()
    return data

In [3]:
def generate_image(prompt, api_key):
    """Generates an image based on the prompt and returns the image ID."""
    data = {"prompt": prompt}
    response_data = send_request('POST', '/items/images/', api_key, data)
    return response_data['data']['id']

In [4]:
def get_image_status(image_id, api_key):
    """Retrieves the status of the image generation."""
    response_data = send_request('GET', f"/items/images/{image_id}", api_key)
    return response_data['data']

In [5]:
def wait_for_image(image_id, api_key):
    """Waits for the image to be generated, checking the status every 5 seconds."""
    while True:
        image_data = get_image_status(image_id, api_key)
        if image_data['status'] in ['completed', 'failed']:
            return image_data
        else:
            print(f"Image generation in progress. Status: {image_data['status']}")
            time.sleep(5)

def download_images(image_data, download_path, selection_path):
    """Downloads the images to specified paths."""
    if not image_data or 'upscaled_urls' not in image_data:
        print("No image data provided for download.")
        return

    urls = image_data['upscaled_urls']
    os.makedirs(download_path, exist_ok=True)
    os.makedirs(selection_path, exist_ok=True)

    selected_image_url = random.choice(urls)
    for url in urls:
        file_name = url.split('/')[-1]
        file_path = os.path.join(selection_path if url == selected_image_url else download_path, file_name)

        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"Image downloaded successfully: {file_path}")
        else:
            print(f"Error downloading image from {url}: {response.status_code}")

In [13]:
def download_images(image_data, download_path):
    """Downloads the images to the specified path."""
    if not image_data or 'upscaled_urls' not in image_data:
        print("No image data provided for download.")
        return

    urls = image_data['upscaled_urls']
    os.makedirs(download_path, exist_ok=True)

    for url in urls:
        file_name = url.split('/')[-1]
        file_path = os.path.join(download_path, file_name)

        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"Image downloaded successfully: {file_path}")
        else:
            print(f"Error downloading image from {url}: {response.status_code}")


In [7]:
# Example usage
api_key = '5tE5Zjk4iIPFRituOqlCbbNSOp74TW6J'
prompt = "a photorealistic image of a tsunami"

image_id = generate_image(prompt, api_key)
image_data = wait_for_image(image_id, api_key)

download_path = "downloads"
selection_path = "selection"
download_images(image_data, download_path, selection_path)


Image generation in progress. Status: pending
Image generation in progress. Status: pending
Image generation in progress. Status: in-progress
Image generation in progress. Status: in-progress
Image generation in progress. Status: in-progress
Image generation in progress. Status: in-progress
Image generation in progress. Status: in-progress
Image generation in progress. Status: in-progress
Completed image details:
{'id': '2620f9a6-f109-46b0-bed9-cf9e7f01b04a', 'prompt': 'a photorealistic image of a tsunami', 'results': '43aef7b2-f76c-4a04-950d-2f2590cbe1b5', 'user_created': '988dd75e-d15c-4938-a763-c5d2d5e6ea4c', 'date_created': '2024-01-01T10:22:01.938Z', 'status': 'completed', 'progress': None, 'url': 'https://cl.imagineapi.dev/assets/43aef7b2-f76c-4a04-950d-2f2590cbe1b5/43aef7b2-f76c-4a04-950d-2f2590cbe1b5.png', 'error': None, 'upscaled_urls': ['https://cl.imagineapi.dev/assets/bad1e2a9-9cb3-484b-bf52-bde7bc094ebe/bad1e2a9-9cb3-484b-bf52-bde7bc094ebe.png', 'https://cl.imagineapi.dev/

In [15]:

download_path = "downloads"
download_images(image_data, download_path)


Image downloaded successfully: downloads/bad1e2a9-9cb3-484b-bf52-bde7bc094ebe.png
Image downloaded successfully: downloads/beb05a11-2b0b-47e5-aac4-f182ef532924.png
Image downloaded successfully: downloads/28a888a9-1810-4e3c-ad91-5bbd6ce3d57e.png
Image downloaded successfully: downloads/7fbffbb1-b378-4a52-beb1-8658c4c8baed.png
